In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt
from scipy.optimize import minimize

In [ ]:
"""
from sklearn.metrics import 

model_classes = ["LightGBM", "XGBoost", "CatBoost"]
target_cols = ['sea_water_temperature', 'sea_water_practical_salinity']

for model_name in model_classes:
    for target_col in target_cols:

        file_path = f'/content/drive/MyDrive/Colab Notebooks/Batch/{model_name}_{target_col}_prediction_whole.csv'


        data = pd.read_csv(file_path)


        actual = data[f'actual_{target_col}']
        predicted = data[f'predicted_{target_col}']

        r2 = r2_score(actual, predicted)
        print(f"Model: {model_name}, {target_col} - R^2: {r2}")
"""

Model: LightGBM, sea_water_temperature - R^2: 0.833878173102925
Model: LightGBM, sea_water_practical_salinity - R^2: 0.8940496966978438
Model: XGBoost, sea_water_temperature - R^2: 0.8377575748260944
Model: XGBoost, sea_water_practical_salinity - R^2: 0.9009908462658189
Model: CatBoost, sea_water_temperature - R^2: 0.8541286243720079
Model: CatBoost, sea_water_practical_salinity - R^2: 0.9021902450787243


In [ ]:
temp_preds1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/LightGBM_sea_water_temperature_prediction_whole_2050.csv')
temp_preds2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/XGBoost_sea_water_temperature_prediction_whole_2050.csv')
temp_preds3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/CatBoost_sea_water_temperature_prediction_whole_2050.csv')


sal_preds1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/LightGBM_sea_water_practical_salinity_prediction_whole_2050.csv')
sal_preds2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/XGBoost_sea_water_practical_salinity_prediction_whole_2050.csv')
sal_preds3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/CatBoost_sea_water_practical_salinity_prediction_whole_2050.csv')


actual_temp = temp_preds1['actual_sea_water_temperature']
actual_salinity = sal_preds1['actual_sea_water_practical_salinity']
temp_predictions = pd.concat([temp_preds1['predicted_sea_water_temperature'],
                              temp_preds2['predicted_sea_water_temperature'],
                              temp_preds3['predicted_sea_water_temperature']], axis=1)

salinity_predictions = pd.concat([sal_preds1['predicted_sea_water_practical_salinity'],
                                  sal_preds2['predicted_sea_water_practical_salinity'],
                                  sal_preds3['predicted_sea_water_practical_salinity']], axis=1)




In [ ]:

temp_preds1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/LightGBM_sea_water_temperature_prediction_2050.csv')
temp_preds2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/XGBoost_sea_water_temperature_prediction_2050.csv')
temp_preds3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/CatBoost_sea_water_temperature_prediction_2050.csv')

sal_preds1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/LightGBM_sea_water_practical_salinity_prediction_2050.csv')
sal_preds2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/XGBoost_sea_water_practical_salinity_prediction_2050.csv')
sal_preds3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Batch/CatBoost_sea_water_practical_salinity_prediction_2050.csv')


temp_predictions = pd.concat([temp_preds1['predicted_sea_water_temperature'],
                              temp_preds2['predicted_sea_water_temperature'],
                              temp_preds3['predicted_sea_water_temperature']], axis=1)

salinity_predictions = pd.concat([sal_preds1['predicted_sea_water_practical_salinity'],
                                  sal_preds2['predicted_sea_water_practical_salinity'],
                                  sal_preds3['predicted_sea_water_practical_salinity']], axis=1)
additional_columns = temp_preds1[['date', 'latitude', 'longitude', 'depth']]



weights_temp = [0.2766, 0.0931, 0.6303]
weights_sal = [0.276, 0.2838, 0.4402]


ensemble_temp = temp_predictions.dot(weights_temp)
ensemble_salinity = salinity_predictions.dot(weights_sal)

ensemble_predictions = pd.DataFrame({
    'date': additional_columns['date'],
    'latitude': additional_columns['latitude'],
    'longitude': additional_columns['longitude'],
    'depth': additional_columns['depth'],
    'ensemble_temperature': ensemble_temp,
    'ensemble_salinity': ensemble_salinity
})


ensemble_predictions.to_csv('/content/drive/MyDrive/Colab Notebooks/Batch/ensemble_predictions_2050.csv', index=False)

print("Ensemble predictions saved to CSV.")


Ensemble predictions saved to CSV.


In [ ]:
def calculate_rmse(weights, predictions, actuals):
    combined_prediction = np.average(predictions, axis=1, weights=weights)
    return sqrt(mean_squared_error(actuals, combined_prediction))
initial_weights = [1/3, 1/3, 1/3] 
bounds = [(0, 1), (0, 1), (0, 1)] 
result_temp = minimize(calculate_rmse, initial_weights, args=(temp_predictions, actual_temp), method='SLSQP', bounds=bounds)
optimized_weights_temp = result_temp.x


In [ ]:
result_salinity = minimize(calculate_rmse, initial_weights, args=(salinity_predictions, actual_salinity), method='SLSQP', bounds=bounds)
optimized_weights_salinity = result_salinity.x
final_predictions_temp = np.average(temp_predictions, axis=1, weights=optimized_weights_temp)
final_predictions_salinity = np.average(salinity_predictions, axis=1, weights=optimized_weights_salinity)
final_results = pd.DataFrame({
    'date': temp_preds1['date'],
    'actual_sea_water_temperature': actual_temp,
    'predicted_sea_water_temperature': final_predictions_temp,
    'actual_sea_water_practical_salinity': actual_salinity,
    'predicted_sea_water_practical_salinity': final_predictions_salinity
})
final_results.to_csv('/content/drive/MyDrive/Colab Notebooks/Batch/final_ensemble_predictions.csv', index=False)



In [ ]:
rmse_temp = sqrt(mean_squared_error(actual_temp, final_predictions_temp))
print(f'RMSE for Temperature Predictions: {rmse_temp}')
rmse_salinity = sqrt(mean_squared_error(actual_salinity, final_predictions_salinity))
print(f'RMSE for Salinity Predictions: {rmse_salinity}')
total_weight_temp = sum(optimized_weights_temp)
normalized_weights_temp = [w / total_weight_temp for w in optimized_weights_temp]
print(f'Normalized Weights for Temperature Models: {normalized_weights_temp}')

total_weight_salinity = sum(optimized_weights_salinity)
normalized_weights_salinity = [w / total_weight_salinity for w in optimized_weights_salinity]
print(f'Normalized Weights for Salinity Models: {normalized_weights_salinity}')
mae_temp = mean_absolute_error(actual_temp, final_predictions_temp)
print(f'MAE for Temperature Predictions: {mae_temp}')
mae_salinity = mean_absolute_error(actual_salinity, final_predictions_salinity)
print(f'MAE for Salinity Predictions: {mae_salinity}')



RMSE for Temperature Predictions: 0.6975153165814829
RMSE for Salinity Predictions: 0.49666833258363685
Normalized Weights for Temperature Models: [0.27659584030545165, 0.09310472521758104, 0.6302994344769672]
Normalized Weights for Salinity Models: [0.27595505902453477, 0.28382096748198066, 0.4402239734934846]
MAE for Temperature Predictions: 0.5326765827112249
MAE for Salinity Predictions: 0.265932561474047


In [ ]:
r2_temp = r2_score(actual_temp, final_predictions_temp)
print(f'R^2 Score for Temperature Predictions: {r2_temp}')
r2_salinity = r2_score(actual_salinity, final_predictions_salinity)
print(f'R^2 Score for Salinity Predictions: {r2_salinity}')

R^2 Score for Temperature Predictions: 0.8594865356133613
R^2 Score for Salinity Predictions: 0.9126115253940816
